# Lesson4 ニューラル翻訳モデルを作ってみよう（Seq2Seq, Attention）

## 目次

- Section3 精度向上Tips
    - 3.1 Attention機構
    - Checkクイズ

## Section3 精度向上Tips

### 3.1 Attention機構

Seq2SeqモデルではLSTMを用いることが一般的ですが、たとえLSTMを用いてもやはり長い系列の予測が相対的に難しいのは事実です。

そもそも、系列が長くなってしまえば、最初に符号化器に入力された情報が復号化器まで伝播することが期待できなくなっていきます。

そこで、より直接的に入力情報を出力時に利用する**Attention機構**と呼ばれるものについて紹介します。

Attention機構では、入力系列の中で各データ（単語）を重視すべき確率を計算し、この重視度合い（attention）を考慮して入力系列を参照します。

なお、この考慮の仕方でAttention機構は大きく二つに分類されます。

* soft attention：attentionの分布による符号化器出力の**期待値**を復号化器出力に利用

* hard attention：attentionの分布からの**サンプリング**によって選択した符号化器出力を復号化器出力に利用

ここでは、比較的実装が容易なsoft attentionに絞って解説します。

<img src="figures/attention.png" width=50%>
出典：T. Luong, H. Pham, and C.D. Manning, "Effective Approaches to Attentionbased Neural Machine Translation," 2015. ( http://aclweb.org/anthology/D15-1166 )

(以下では上付き添え字の$(e)$で符号化器の、$(d)$で復号化器のベクトルであることを表現します。)

復号化器のある生の出力ベクトル$h_t^{(d)}$に注目しましょう。この$h_t^{(d)}$を符号化器の状態を考慮した$\tilde{h}_t$に変換することを考えます。

そこでまず、$h_t^{(d)}$と符号化器の出力ベクトルとの関係を数値化し、どれを重視すればよいか測りたいと思います。

この数値化を担うのが、**スコア関数**です。スコア関数には以下のような関数が用いられます。

$$\omega_{ts} = \Omega(h_s^{(e)},h_t^{(d)}) = \begin{cases}h_s^{(e)} \cdot h_t^{(d)} \\ h_s^{(e)} \cdot W h_t^{(d)}\\ v \cdot \tanh \left(W \left[\begin{array}{c} h_s^{(e)} \\ h_t^{(d)}\end{array}\right]\right)\end{cases}$$

このスコア関数によって、1つの符号化器の隠れ状態ベクトルに対して、符号化器の系列長分だけのスコア$\omega_t=\{\omega_{ts}\}_s$が得られることになります。

これらスコアを確率に変換して、Attentionとして利用することを考えます。

確率への変換については、良く行われるように、softmax関数を利用します。

つまり、**Attention** $a_t=\{a_{ts}\}_s$は$$a_{ts}=\frac{\exp(\omega_{ts})}{\sum_s \exp(\omega_{ts})}$$によって決まります。

このAttentionによって、符号化器の隠れ状態ベクトルの重み付き平均をとったもの、$$c_t=\sum_s a_{ts} h_s^{(e)}$$を**文脈ベクトル**と呼びます。

最終的な出力の生成に当たっては、この文脈ベクトルと元の復号化器の状態ベクトルをひとまとめにしたベクトルを、

tanhの活性化関数に通すことで行います。

$$\tilde{h}_t = \tanh \left(W^{(a)} \left[\begin{array}{c} c_t \\ h_t^{(d)}\end{array}\right]\right)$$

例えば、文章を要約するタスクでAttentionを可視化したものが下図です。

このように各出力単語（横軸）に対して、各入力単語（縦軸）に対するAttentionが得られ、相対的な入力の重視度合いが調整されています。

<img src="figures/case_attention.jpg" width=30%>
出典：A.M. Rush, S. Chopra, and J. Weston, "A Neural Attention Model for Abstractive Sentence Summarization," 2015. ( https://arxiv.org/abs/1509.00685 )

### Checkクイズ

* 問題1
    * Attention機構の説明として最も正しいものを選びなさい
* 選択肢
  1. 短い系列の場合にはAttention機構は無力である  
  2. ピュアなSeq2Seqモデルでは使用していなかった、復号化器の出力を利用する  
  3. Soft attentionによるSeq2Seqモデルでは、文脈ベクトル（Attentionによる符号化器出力の加重平均）にtanh関数を適用したものが出力である  
  4. Soft attentionによるSeq2Seqモデルでは、符号化器と復号化器の出力の関係をスコア関数で数値化し、softmax関数を適用したものがAttentionである